# Given a stream. Calculate the running mean

#### Whole stream

```C++
class Stream {
private:
    int cum_sum;
    int N;
public:
    Stream(): cum_sum(0), N(0) {}
    double get_mean(int new_val) {
        cum_sum += new_val;
        N += 1;
        return static_cast<double>(cum_sum) / N;
    }
};
```

#### Windows - size K

```Cpp
class Stream {
private:
    int cum_sum;
    deque<int> deq;
    int k;
public:
    Stream(int window_size): cum_sum(0), k(window_size) {
        assert(k > 0);
        deq.clear();
    }
    double get_mean(int new_val) {
        // Check if remove back
        if(deq.size() == k) {
            int remove_elem = deq.back();
            deq.pop_back();
            cum_sum -= remove_elem;
        }

        // Enqueue new elem
        deq.push_front(new_val);
        cum_sum += new_val;

        return static_cast<double>(cum_sum) / deq.size();
    }
};
```

# Given a stream. Calculate the running median

#### [Whole stream](https://leetcode.com/problems/find-median-from-data-stream/)

```C++
// Maintain
// min[left_half]max - min[right_half]max
//      left size = right size or left size = right size + 1
//      all left <= all right
class MedianFinder {
private:
    int _N;
    multiset<int> _left_half;
    multiset<int> _right_half;

    void __propagate() {
        // left -> right
        if(_left_half.size() > _right_half.size() + 1) {
            int tmp = *_left_half.rbegin();
            _left_half.erase(_left_half.lower_bound(tmp));

            _right_half.insert(tmp);
        }

        // right -> left
        else if(_left_half.size() < _right_half.size()) {
            int tmp = *_right_half.begin();
            _right_half.erase(_right_half.lower_bound(tmp));

            _left_half.insert(tmp);
        }
    }
public:
    MedianFinder() {
        _left_half.clear();
        _right_half.clear();
        _N = 0;
    }

    void addNum(int num) {
        if(_right_half.size() == 0 || num < *_right_half.begin()) {
            _left_half.insert(num);
        } else {
            _right_half.insert(num);
        }

        _N += 1;
        __propagate();
    }

    double findMedian() {
        // odd
        if(_N % 2 == 1) {
            int m = *_left_half.rbegin();
            return static_cast<double>(m);
        }

        // even
        int m1 = *_left_half.rbegin();
        int m2 = *_right_half.begin();
        return static_cast<double>(m1 + m2) / 2.0;
    }
};
```

#### [Windows - size K](https://leetcode.com/problems/sliding-window-median/)

```Cpp
class Solution {
public:
    vector<double> medianSlidingWindow(vector<int>&A, int k) {
        multiset<int> window(A.begin(), A.begin() + k);

        // keep an iterator pointing to the middle value
        auto m2 = next(window.begin(), k/2);

        vector<double> ans;
        for(int r=k; ; ++r) {
            int l = r - k;

            // update ans
            (k % 2 == 1) ?
                ans.push_back(static_cast<double>(*m2)) : \
                ans.push_back((static_cast<double>(*m2) + static_cast<double>(*prev(m2, 1))) / 2.0);
            if(r == A.size()) return ans;

            // insert
            window.insert(A[r]);
            if(A[r] < *m2) m2 = prev(m2); // keep m2 update

            // remove
            if(A[l] <= *m2) m2 = next(m2);
            window.erase(window.lower_bound(A[l])); // keep m2 update
        }
        return ans;
    }
};
```

- Add deque to simulate stream processing

```Cpp
class Solution {
public:
    vector<double> medianSlidingWindow(vector<int>&A, int k) {
        deque<int> window(A.begin(), A.begin() + k);
        multiset<int> ordered_window(A.begin(), A.begin() + k);

        // keep an iterator pointing to the middle value
        auto m2 = next(ordered_window.begin(), k/2);

        int data_point = -1;
        vector<double> ans;
        for(int i=k; ; ++i) {
            // update ans
            (k % 2 == 1) ?
                ans.push_back(static_cast<double>(*m2)) : \
                ans.push_back((static_cast<double>(*m2) + static_cast<double>(*prev(m2, 1))) / 2.0);
            if(i == A.size()) return ans;

            // insert
            data_point = A[i];
            window.push_back(data_point);

            ordered_window.insert(data_point);
            if(data_point < *m2) m2 = prev(m2); // keep m2 update


            // remove
            data_point = window.front(); window.pop_front();

            if(data_point <= *m2) m2 = next(m2);
            ordered_window.erase(ordered_window.lower_bound(data_point)); // keep m2 update
        }
        return ans;
    }
};
```